# Дообучение RuBert для определения спам сообщений

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from langdetect import detect  # для создания выборок сообщений на русском языке

In [3]:
import torch

In [4]:
from copy import deepcopy

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [6]:
from datasets import Dataset

In [7]:
from torch.utils.data import DataLoader

In [8]:
from transformers import TrainingArguments, Trainer

In [77]:
from transformers import pipeline

In [78]:
import sys
sys.path.append('../utils')
from preprocessing import preprocess_text

## Чтение обработанного датасета

In [25]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [26]:
df.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


### Датасет без обработки

In [27]:
only_text = deepcopy(df[['text', 'label']])

In [28]:
dataset = only_text.to_dict('records')

In [29]:
ru_dataset = []

In [30]:
for d in dataset:
    if not(1072 <= ord(d['text'][0].lower()) <= 1103):
        continue
    ru_dataset.append({"text": d['text'], "label": d['label']})

In [31]:
ru_dataset

[{'text': 'Добрый день! Отличается ли перечень необходимых документов для иностранных граждан?',
  'label': 0},
 {'text': 'Узбекистан. Рассматриваются обе формы', 'label': 0},
 {'text': 'Здравствуйте, а как проходит поступление после колледжа? Внутренний экзамен или только ЕГЭ? И возможно ли всё ещё поступление сразу на второй курс?',
  'label': 0},
 {'text': 'Спасибо большое за ответ!', 'label': 0},
 {'text': 'Здравствуйте, а когда будет день открытых дверей и во сколько?',
  'label': 0},
 {'text': 'Спасибо', 'label': 0},
 {'text': 'Здравствуйте, необходимо для посещения дня открытым дверей где-то регистрироваться',
  'label': 0},
 {'text': 'Добрый день! Да, необходимо пройти быструю регистрацию на сайте https://info.stankin.ru/dod',
  'label': 0},
 {'text': 'Лучшая специальность! Жаль с моими баллами не пройти(', 'label': 0},
 {'text': 'Было бы неплохо, если бы прикладывались ссылки для просмотра результатов экзаменов.)',
  'label': 0},
 {'text': 'Здравствуйте. Не смог найти списки п

### Датасет с обработкой

In [33]:
only_text_p = deepcopy(df[['text_preprocessed', 'label']])

In [34]:
dataset_p = only_text_p.to_dict('records')

In [35]:
ru_dataset_p = []

In [37]:
for d in dataset_p:
    if not(1072 <= ord(d['text_preprocessed'][0].lower()) <= 1103):
        continue
    ru_dataset_p.append({"text": d['text_preprocessed'], "label": d['label']})

In [38]:
len(dataset), len(ru_dataset), len(ru_dataset_p)

(25013, 24221, 24375)

## Дообучение RuBert

### Загрузка модели и токенайзера

In [41]:
MODEL_NAME = "cointegrated/rubert-tiny2"
# MODEL_NAME = "DeepPavlov/rubert-base-cased"

In [42]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
rb_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)  # num_labels=2 для бинарной классификации

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготовка выборок

In [43]:
def tokenize_function(examples):  # Почти лямбда функция для токенизации данных
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

#### Без предобработки

Загрузка датасета без предобработки

In [44]:
ru_df = pd.json_normalize(ru_dataset)

Преобразование в формат Hugging Face

In [45]:
dataset_hf = Dataset.from_pandas(ru_df)

Разделение на обучающую и тестовую выборки

In [46]:
rb_train_test_split = dataset_hf.train_test_split(test_size=0.2, seed=42)
rb_train_dataset = rb_train_test_split['train']
rb_test_dataset = rb_train_test_split['test']

In [47]:
ru_df_spam = ru_df.loc[ru_df['label'] == 1]

In [48]:
ru_df_spam

,text,label
24070,Только что выиграл 26000 рублей в новом телегр...,1
24071,ЗАПУСТИЛИ ОФИЦИАЛЬНОЕ КАЗИНО С ОГРОМНЫМ ШАНСОМ...,1
24072,"Заработал 800$ в новом телеграм казино, кому и...",1
24073,"Выиграл 23000 рублей, играю в этом казике @Big...",1
24074,"Выиграл 4500$ в олимпусе, играю тут @BigCasino...",1
...,...,...
24216,в стиме началась раздача 10$ в честь дня рожде...,1
24217,Пиздец чекай https://t.me/SupeRRC0ntent_bot,1
24218,"Прикинь чё нашёл, заходи быстрее discord.gg/cV...",1
24219,"Привет, ты уже забрал подарок 20$ от стима? [s...",1


Токенизация данных

In [54]:
rb_tokenized_train = rb_train_dataset.map(tokenize_function, batched=True)
rb_tokenized_test = rb_test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/19376 [00:00<?, ? examples/s]

Map:   0%|          | 0/4845 [00:00<?, ? examples/s]

#### С предобработкой

Загрузка датасета c предобработкой

In [49]:
ru_df_p = pd.json_normalize(ru_dataset_p)

Преобразование в формат Hugging Face

In [50]:
dataset_hf_p = Dataset.from_pandas(ru_df_p)

Разделение на обучающую и тестовую выборки

In [51]:
rb_train_test_split_p = dataset_hf_p.train_test_split(test_size=0.2, seed=42)
rb_train_dataset_p = rb_train_test_split_p['train']
rb_test_dataset_p = rb_train_test_split_p['test']

In [52]:
ru_df_spam_p = ru_df_p.loc[ru_df_p['label'] == 1]

In [53]:
ru_df_spam_p

,text,label
24172,официальное телеграм казино играй прямо в боте,1
24173,топ 4 казино с самыми высокими шансами выигрыш...,1
24174,только что выиграл 26000 рублей в новом телегр...,1
24175,запустили официальное казино с огромным шансом...,1
24176,заработал 800 в новом телеграм казино кому инт...,1
...,...,...
24370,в стиме началась раздача 10 в честь дня рожден...,1
24371,пиздец чекай [LINK],1
24372,прикинь чё нашёл заходи быстрее [LINK],1
24373,привет ты уже забрал подарок 20 от стима [LINK],1


Токенизация данных

In [55]:
rb_tokenized_train_p = rb_train_dataset_p.map(tokenize_function, batched=True)
rb_tokenized_test_p = rb_test_dataset_p.map(tokenize_function, batched=True)

Map:   0%|          | 0/19500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4875 [00:00<?, ? examples/s]

### Подготовка trainer'а

In [58]:
train_loader = DataLoader(rb_tokenized_train, batch_size=16, shuffle=True)
test_loader = DataLoader(rb_tokenized_test, batch_size=16)

In [59]:
train_loader_p = DataLoader(rb_tokenized_train_p, batch_size=16, shuffle=True)
test_loader_p = DataLoader(rb_tokenized_test_p, batch_size=16)

In [61]:
rb_training_args = TrainingArguments(
    output_dir="rb_results",  # Папка для сохранения модели
    eval_strategy="epoch",  # Оценка модели на каждом эпохе
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Количество эпох
    weight_decay=0.01,
    logging_dir="rb_logs",  # Логи обучения
    logging_steps=10,
    save_total_limit=2,  # Сохранение только последних двух моделей
)

In [63]:
rb_trainer = Trainer(
    model=rb_model,
    args=rb_training_args,
    train_dataset=rb_tokenized_train,
    eval_dataset=rb_tokenized_test,
    processing_class=tokenizer
)

In [71]:
rb_trainer_p = Trainer(
    model=rb_model,
    args=rb_training_args,
    train_dataset=rb_tokenized_train_p,
    eval_dataset=rb_tokenized_test_p,
    processing_class=tokenizer
)

### Запуск дообучения

#### Перемещение модели на GPU

In [66]:
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

# Переместите модель на GPU
rb_model.to(device)

Используемое устройство: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-

#### Тренировка

##### Без предобработки данных

In [67]:
rb_trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000200,0.002033
2,0.000600,0.000699
3,0.000100,0.000615


TrainOutput(global_step=3633, training_loss=0.007789298103766402, metrics={'train_runtime': 369.5209, 'train_samples_per_second': 157.306, 'train_steps_per_second': 9.832, 'total_flos': 428647903100928.0, 'train_loss': 0.007789298103766402, 'epoch': 3.0})

In [68]:
rb_trainer.evaluate()

{'eval_loss': 0.0006148326792754233,
 'eval_runtime': 14.8641,
 'eval_samples_per_second': 325.953,
 'eval_steps_per_second': 20.385,
 'epoch': 3.0}

Сохранение модели

In [69]:
rb_model.save_pretrained("../models/finetuned_rubert_tiny2")
tokenizer.save_pretrained("../models/finetuned_rubert_tiny2")

('../models/finetuned_rubert_tiny2\\tokenizer_config.json',
 '../models/finetuned_rubert_tiny2\\special_tokens_map.json',
 '../models/finetuned_rubert_tiny2\\vocab.txt',
 '../models/finetuned_rubert_tiny2\\added_tokens.json',
 '../models/finetuned_rubert_tiny2\\tokenizer.json')

##### С предобработкой данных

In [72]:
rb_trainer_p.train()

Epoch,Training Loss,Validation Loss
1,0.003600,0.007194
2,0.000000,0.005298
3,0.000000,0.005155


TrainOutput(global_step=3657, training_loss=0.0018951208413552619, metrics={'train_runtime': 375.2746, 'train_samples_per_second': 155.886, 'train_steps_per_second': 9.745, 'total_flos': 431391108096000.0, 'train_loss': 0.0018951208413552619, 'epoch': 3.0})

In [73]:
rb_trainer_p.evaluate()

{'eval_loss': 0.005155482329428196,
 'eval_runtime': 15.0809,
 'eval_samples_per_second': 323.257,
 'eval_steps_per_second': 20.224,
 'epoch': 3.0}

Сохранение модели

In [74]:
rb_model.save_pretrained("../models/finetuned_rubert_tiny2_p")
tokenizer.save_pretrained("../models/finetuned_rubert_tiny2_p")

('../models/finetuned_rubert_tiny2_p\\tokenizer_config.json',
 '../models/finetuned_rubert_tiny2_p\\special_tokens_map.json',
 '../models/finetuned_rubert_tiny2_p\\vocab.txt',
 '../models/finetuned_rubert_tiny2_p\\added_tokens.json',
 '../models/finetuned_rubert_tiny2_p\\tokenizer.json')

## Оценка качества моделей

In [75]:
# Тестовые сообщения
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!"
]

In [79]:
classifier = pipeline("text-classification", model="../models/finetuned_rubert_tiny2", tokenizer="../models/finetuned_rubert_tiny2")

Device set to use cuda:0


In [80]:
classifier_p = pipeline("text-classification", model="../models/finetuned_rubert_tiny2_p", tokenizer="../models/finetuned_rubert_tiny2_p")

Device set to use cuda:0


In [86]:
for message in test_messages:
    result = classifier(message)
    print(message)
    print(result)
    print()

Это честное сообщение от пользователя.
[{'label': 'LABEL_0', 'score': 0.9999589920043945}]

🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
[{'label': 'LABEL_1', 'score': 0.999045193195343}]

Зарабатывай миллионы **онлайн** прямо сейчас!
[{'label': 'LABEL_1', 'score': 0.9961320161819458}]

Работа на дому, легкий доход. Пиши в личку!
[{'label': 'LABEL_0', 'score': 0.9998481273651123}]

Привет! Как дела? У меня всё отлично.
[{'label': 'LABEL_0', 'score': 0.9999611377716064}]

Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
[{'label': 'LABEL_1', 'score': 0.9952722191810608}]

Disco

In [88]:
for message in test_messages:
    # message = preprocess_text(message)
    result = classifier_p(message)
    print(message)
    print(result)
    print()

Это честное сообщение от пользователя.
[{'label': 'LABEL_0', 'score': 0.999996542930603}]

🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
[{'label': 'LABEL_1', 'score': 0.9999890327453613}]

Зарабатывай миллионы **онлайн** прямо сейчас!
[{'label': 'LABEL_1', 'score': 0.9999821186065674}]

Работа на дому, легкий доход. Пиши в личку!
[{'label': 'LABEL_1', 'score': 0.9511370062828064}]

Привет! Как дела? У меня всё отлично.
[{'label': 'LABEL_0', 'score': 0.9999966621398926}]

Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
[{'label': 'LABEL_1', 'score': 0.9999837875366211}]

Disco